In [471]:
import pandas
import numpy
from scipy import stats

## Hybrid sample

In [472]:
filepath = 'D:/BG/Data/Processing/4_data_hybrid.txt'
hybrid = pandas.read_csv(filepath, sep='\t')
print(hybrid.shape[0],'job postings in hybrid sample')
hybrid.columns

212821 job postings in hybrid sample


Index(['BGTJobId', 'JobDate', 'CleanTitle', 'CanonTitle', 'SOC', 'SOCName',
       'ONET', 'ONETName', 'Employer', 'Sector', 'SectorName', 'NAICS3',
       'NAICS4', 'NAICS5', 'NAICS6', 'City', 'County', 'State', 'Lat', 'Lon',
       'Edu', 'MaxEdu', 'Degree', 'MaxDegree', 'Exp', 'MaxExp', 'MinSalary',
       'MaxSalary', 'MinHrlySalary', 'MaxHrlySalary', 'PayFrequency',
       'SalaryType', 'JobHours', 'SOC2', 'SOC4', 'SOC5', 'SK',
       'Employer_clean', 'n_terms', 'SK_without', 'TECH', 'occupation',
       'firm1', 'firm2', 'firm', 'plant', 'plant_all', 'plant_5',
       'development', 'materials', 'design', 'inventory', 'tooling',
       'automation', 'production', 'maintenance', 'technical',
       'administrative', 'management', 'finance', 'business', 'data',
       'software', 'office', 'ml', 'cognitive', 'creativity', 'social',
       'character', 'customer', 'writing', 'nonroutine analytic',
       'nonroutine manual', 'routine cognitive', 'routine manual',
       'sequential

In [473]:
mydf = hybrid.copy()
mydf['Count of Terms']=[len(job.split(',')) for job in mydf.SK_without.tolist()]
annotation = ['nonroutine analytic','nonroutine manual','routine cognitive','routine manual','reciprocal','sequential']
for a in annotation:
  mydf[a] = mydf[a]/mydf['Count of Terms']

## Statistics for all occupations

In [474]:
df = mydf[['TECH'] + annotation]

t = []
for a in annotation:
  t.append([a,stats.ttest_ind(df.loc[df.TECH=='AM',a],df.loc[df.TECH=='TM',a],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['annotation','p-val'])
tdf.set_index('annotation',inplace=True) 

mymean = df.groupby(by=['TECH']).mean().T
mymean.index.name = 'annotation'
mymean['AM-TM'] = mymean['AM'] - mymean['TM']

mymean['N-AM'] = N = len(mydf.loc[mydf.TECH=='AM'])
mymean['N-TM'] = N = len(mydf.loc[mydf.TECH=='TM'])

table1 = pandas.merge(left=mymean,right=tdf,how='inner',left_on=['annotation'],right_on=['annotation'])
table1['statistics'] = 'Mean'
table1.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['TECH']).std(ddof=0).T
mysd.index.name = 'annotation'
mysd.reset_index(inplace=True)
mysd['statistics'] = 'SD'
mysd.set_index(['annotation','statistics'],inplace=True)
mysd['p-val'] = numpy.nan
table1 = pandas.concat([table1,mysd],axis=0).sort_index(kind='merge').reindex(annotation,axis=0,level='annotation')
table1.columns = pandas.MultiIndex.from_product([['All'],table1.columns])

postings = mydf.groupby(['TECH']).nunique()[['BGTJobId']].T
postings.rename_axis([None],axis=1,inplace=True)
postings.columns = pandas.MultiIndex.from_product([['All'],postings.columns])
postings.index = pandas.MultiIndex.from_product([['Number of job postings'],['']])
plants = mydf.groupby(['TECH']).nunique()[['plant']].T
plants.rename_axis([None],axis=1,inplace=True)
plants.columns = pandas.MultiIndex.from_product([['All'],plants.columns])
plants.index = pandas.MultiIndex.from_product([['Number of plants'],['']])
table1 = pandas.concat([table1,postings,plants],axis=0)
table1

All                              \
                                            AM     AM-TM    N-AM      N-TM   
annotation             statistics                                            
nonroutine analytic    Mean           0.111699  0.030014  4638.0  208183.0   
                       SD             0.102880       NaN     NaN       NaN   
nonroutine manual      Mean           0.006377 -0.004985  4638.0  208183.0   
                       SD             0.026801       NaN     NaN       NaN   
routine cognitive      Mean           0.002474 -0.001511  4638.0  208183.0   
                       SD             0.013493       NaN     NaN       NaN   
routine manual         Mean           0.052322  0.007693  4638.0  208183.0   
                       SD             0.079035       NaN     NaN       NaN   
reciprocal             Mean           0.007290  0.000666  4638.0  208183.0   
                       SD             0.024765       NaN     NaN       NaN   
sequential             Mean           0.052971 -0.022685  4638.0  208183.0   
                       SD             0.072637       NaN     NaN       NaN   
Number of job postings             4638.000000       NaN     NaN       NaN   
Number of plants                    319.000000       NaN     NaN       NaN   

                                                                
                                              TM         p-val  
annotation             statistics                               
nonroutine analytic    Mean             0.081685  9.561266e-83  
                       SD               0.103441           NaN  
nonroutine manual      Mean             0.011362  1.612789e-34  
                       SD               0.041327           NaN  
routine cognitive      Mean             0.003985  1.303517e-13  
                       SD               0.021150           NaN  
routine manual         Mean             0.044630  6.325611e-11  
                       SD               0.081659           NaN  
reciprocal             Mean             0.006624  7.034271e-02  
                       SD               0.025319           NaN  
sequential             Mean             0.075655  3.993299e-93  
                       SD               0.090139           NaN  
Number of job postings             208183.000000           NaN  
Number of plants                      319.000000           NaN

## Statistics by occupation

In [475]:
occupations = ['Manager','Engineer','Technician','Operator']
df = mydf[['TECH','occupation'] + annotation]

t = []
for a in annotation:
  for o in occupations:
    t.append([a,o,stats.ttest_ind(df.loc[(df.TECH=='AM') & (df.occupation==o),a],df.loc[(df.TECH=='TM') & (df.occupation==o),a],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['annotation','occupations','p-val'])
tdf.set_index(['annotation','occupations'],inplace=True)

mymean = df.groupby(by=['occupation','TECH']).mean().unstack(level=0).T
mymean.index.names = ['annotation','occupations']
mymean['AM-TM'] = mymean['AM'] - mymean['TM']

mymean.loc[pandas.IndexSlice[:,'Manager'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Manager')])
mymean.loc[pandas.IndexSlice[:,'Engineer'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Engineer')])
mymean.loc[pandas.IndexSlice[:,'Technician'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Technician')])
mymean.loc[pandas.IndexSlice[:,'Operator'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Operator')])
mymean.loc[pandas.IndexSlice[:,'Manager'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Manager')])
mymean.loc[pandas.IndexSlice[:,'Engineer'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Engineer')])
mymean.loc[pandas.IndexSlice[:,'Technician'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Technician')])
mymean.loc[pandas.IndexSlice[:,'Operator'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Operator')])

table2 = pandas.merge(left=mymean,right=tdf,how='inner',left_on=['annotation','occupations'],right_on=['annotation','occupations'])
table2['statistics'] = 'Mean'
table2.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['occupation','TECH']).std(ddof=0).unstack(level=0).T
mysd.index.names = ['annotation','occupations']
mysd['statistics'] = 'SD'
mysd.set_index('statistics',append=True,inplace=True)
mysd['p-val'] = numpy.nan

table2 = pandas.concat([table2,mysd],axis=0).sort_index(kind='merge').reindex(annotation,axis=0,level='annotation').unstack('occupations').swaplevel(0,1,axis=1).reindex(occupations,axis=1,level=0)
table2.rename_axis([None,None],axis=1,inplace=True)

postings = mydf.groupby(['occupation','TECH']).nunique()[['BGTJobId']].reindex(occupations, level='occupation').T
postings.rename_axis([None,None],axis=1,inplace=True)
postings.index = pandas.MultiIndex.from_product([['Number of job postings'],['']])
plants = mydf.groupby(['occupation','TECH']).nunique()[['plant']].reindex(occupations, level='occupation').T
plants.rename_axis([None,None],axis=1,inplace=True)
plants.index = pandas.MultiIndex.from_product([['Number of plants'],['']])
table2 = pandas.concat([table2,postings,plants],axis=0).reindex(occupations,axis=1,level=0)
table2

Manager                            \
                                           AM     AM-TM   N-AM     N-TM   
annotation             statistics                                         
nonroutine analytic    Mean          0.074100  0.014442  981.0  88819.0   
                       SD            0.086409       NaN    NaN      NaN   
nonroutine manual      Mean          0.000658 -0.002921  981.0  88819.0   
                       SD            0.007233       NaN    NaN      NaN   
routine cognitive      Mean          0.001233 -0.000231  981.0  88819.0   
                       SD            0.009065       NaN    NaN      NaN   
routine manual         Mean          0.052769  0.028176  981.0  88819.0   
                       SD            0.086721       NaN    NaN      NaN   
reciprocal             Mean          0.014263  0.004018  981.0  88819.0   
                       SD            0.035819       NaN    NaN      NaN   
sequential             Mean          0.077999 -0.022152  981.0  88819.0   
                       SD            0.084730       NaN    NaN      NaN   
Number of job postings             981.000000       NaN    NaN      NaN   
Number of plants                   166.000000       NaN    NaN      NaN   

                                                                  Engineer  \
                                             TM         p-val           AM   
annotation             statistics                                            
nonroutine analytic    Mean            0.059658  2.306313e-07     0.145064   
                       SD              0.077532           NaN     0.101577   
nonroutine manual      Mean            0.003579  9.580863e-32     0.003828   
                       SD              0.021279           NaN     0.016832   
routine cognitive      Mean            0.001465  4.281779e-01     0.002354   
                       SD              0.011316           NaN     0.011899   
routine manual         Mean            0.024593  4.220755e-23     0.036584   
                       SD              0.050584           NaN     0.053395   
reciprocal             Mean            0.010245  4.904876e-04     0.005181   
                       SD              0.030729           NaN     0.019372   
sequential             Mean            0.100151  1.232572e-15     0.045746   
                       SD              0.091638           NaN     0.065525   
Number of job postings             88819.000000           NaN  2704.000000   
Number of plants                     311.000000           NaN   274.000000   

                                                              ... Technician  \
                                      AM-TM    N-AM     N-TM  ...       N-AM   
annotation             statistics                             ...              
nonroutine analytic    Mean        0.013902  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
nonroutine manual      Mean       -0.002340  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
routine cognitive      Mean       -0.002145  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
routine manual         Mean        0.001360  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
reciprocal             Mean        0.000814  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
sequential             Mean       -0.012043  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
Number of job postings                  NaN     NaN      NaN  ...        NaN   
Number of plants                        NaN     NaN      NaN  ...        NaN   

                                                                        \


## Final table

In [476]:
table = pandas.merge(left=table1,right=table2,how='inner',left_on=['annotation','statistics'],right_on=['annotation','statistics'])
table[('Manager','Within')] = (table[('Manager','N-TM')]*table[('Manager','AM-TM')])/table[('All','N-TM')]
table[('Manager','Between')] = (table[('Manager','N-AM')]/table[('All','N-AM')]-table[('Manager','N-TM')]/table[('All','N-TM')])*table[('Manager','AM')]
table[('Engineer','Within')] = (table[('Engineer','N-TM')]*table['Engineer','AM-TM'])/table['All','N-TM']
table[('Engineer','Between')] = (table[('Engineer','N-AM')]/table[('All','N-AM')]-table[('Engineer','N-TM')]/table[('All','N-TM')])*table[('Engineer','AM')]
table[('Technician','Within')] = (table[('Technician','N-TM')]*table['Technician','AM-TM'])/table['All','N-TM']
table[('Technician','Between')] = (table[('Technician','N-AM')]/table[('All','N-AM')]-table[('Technician','N-TM')]/table[('All','N-TM')])*table[('Technician','AM')]
table[('Operator','Within')] = (table[('Operator','N-TM')]*table['Operator','AM-TM'])/table['All','N-TM']
table[('Operator','Between')] = (table[('Operator','N-AM')]/table[('All','N-AM')]-table[('Operator','N-TM')]/table[('All','N-TM')])*table[('Operator','AM')]
table[('All','Within-Check')] = table[('Manager','Within')]+table[('Engineer','Within')]+table[('Technician','Within')]+table[('Operator','Within')]
table[('All','Between')] = table[('Manager','Between')]+table[('Engineer','Between')]+table[('Technician','Between')]+table[('Operator','Between')]
table = numpy.trunc(1000*table)/1000
table[('All','Within')] = table[('All','AM')] - table[('All','TM')] - table['All','Between']
check = table[[('All','Within-Check'),('All','Within')]]
print(check) # because of decimal places; 'Within-Check' is calculated from the occupations, 'Within' is calculated from All to deal with decimal places

mylist = [('All','AM'),('All','TM'),('All','Between'),('All','Within'),('Manager','AM'),('Manager','TM'),('Engineer','AM'),('Engineer','TM'),('Technician','AM'),('Technician','TM'),('Operator','AM'),('Operator','TM')]
table[mylist] = table[mylist].applymap('{:.3f}'.format)

for i in ['All','Manager','Engineer','Technician','Operator']:
  table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')]+'*'
  table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')]+'**'
  table.loc[table[(i,'p-val')]<0.01,(i,'AM')] = table.loc[table[(i,'p-val')]<0.01,(i,'AM')]+'***'

table = table[mylist]
table.loc[pandas.IndexSlice[:,'SD'],:] = table.loc[pandas.IndexSlice[:,'SD'],:].apply(lambda x: '(' + x + ')')
table = table.mask(table == '(nan)',numpy.nan)
table = table.mask(table == 'nan',numpy.nan)
table = table.mask(table == '-0.000','0.000')
table.loc[pandas.IndexSlice['Number of job postings',:],:] = table.loc[pandas.IndexSlice['Number of job postings',:],:].replace(r'.000','',regex=True)
table.loc[pandas.IndexSlice['Number of plants',:],:] = table.loc[pandas.IndexSlice['Number of plants',:],:].replace(r'.000','',regex=True)

table

                                           All       
                                  Within-Check Within
annotation             statistics                    
nonroutine analytic    Mean              0.014  0.015
                       SD                  NaN    NaN
nonroutine manual      Mean             -0.005 -0.005
                       SD                  NaN    NaN
routine cognitive      Mean             -0.001 -0.001
                       SD                  NaN    NaN
routine manual         Mean              0.010  0.010
                       SD                  NaN    NaN
reciprocal             Mean              0.002  0.002
                       SD                  NaN    NaN
sequential             Mean             -0.015 -0.016
                       SD                  NaN    NaN
Number of job postings                     NaN    NaN
Number of plants                           NaN    NaN


All                           \
                                         AM       TM Between  Within   
annotation             statistics                                      
nonroutine analytic    Mean        0.111***    0.081   0.015   0.015   
                       SD           (0.102)  (0.103)     NaN     NaN   
nonroutine manual      Mean        0.006***    0.011   0.000  -0.005   
                       SD           (0.026)  (0.041)     NaN     NaN   
routine cognitive      Mean        0.002***    0.003   0.000  -0.001   
                       SD           (0.013)  (0.021)     NaN     NaN   
routine manual         Mean        0.052***    0.044  -0.002   0.010   
                       SD           (0.079)  (0.081)     NaN     NaN   
reciprocal             Mean          0.007*    0.006  -0.001   0.002   
                       SD           (0.024)  (0.025)     NaN     NaN   
sequential             Mean        0.052***    0.075  -0.007  -0.016   
                       SD           (0.072)  (0.090)     NaN     NaN   
Number of job postings                 4638   208183     NaN     NaN   
Number of plants                        319      319     NaN     NaN   

                                    Manager           Engineer           \
                                         AM       TM        AM       TM   
annotation             statistics                                         
nonroutine analytic    Mean        0.074***    0.059  0.145***    0.131   
                       SD           (0.086)  (0.077)   (0.101)  (0.120)   
nonroutine manual      Mean        0.000***    0.003  0.003***    0.006   
                       SD           (0.007)  (0.021)   (0.016)  (0.028)   
routine cognitive      Mean           0.001    0.001  0.002***    0.004   
                       SD           (0.009)  (0.011)   (0.011)  (0.021)   
routine manual         Mean        0.052***    0.024     0.036    0.035   
                       SD           (0.086)  (0.050)   (0.053)  (0.059)   
reciprocal             Mean        0.014***    0.010   0.005**    0.004   
                       SD           (0.035)  (0.030)   (0.019)  (0.021)   
sequential             Mean        0.077***    0.100  0.045***    0.057   
                       SD           (0.084)  (0.091)   (0.065)  (0.080)   
Number of job postings                  981    88819      2704    78324   
Number of plants                        166      311       274      309   

                                  Technician           Operator           
                                          AM       TM        AM       TM  
annotation             statistics                                         
nonroutine analytic    Mean            0.065    0.062  0.046***    0.025  
                       SD            (0.088)  (0.100)   (0.075)  (0.061)  
nonroutine manual      Mean         0.018***    0.033  0.020***    0.039  
                       SD            (0.047)  (0.060)   (0.050)  (0.075)  
routine cognitive      Mean         0.004***    0.007  0.003***    0.008  
                       SD            (0.018)  (0.030)   (0.021)  (0.033)  
routine manual         Mean         0.100***    0.083  0.093***    0.114  
                       SD            (0.110)  (0.102)   (0.108)  (0.136)  
reciprocal             Mean         0.007***    0.002    0.004*    0.003  
                       SD            (0.026)  (0.015)   (0.019)  (0.017)  
sequential             Mean         0.044***    0.053    0.050*    0.057  
                       SD            (0.061)  (0.087)   (0.081)  (0.092)  
Number of job postings                   457    10791       496    30249  
Number of plants                         104      273       131      300

In [477]:
table.to_excel('Appendix-Table2-share.xlsx')